In [10]:
import pandas as pd
from tqdm.notebook import tqdm 
from multiprocessing import Pool 
from scipy.spatial import distance 
import numpy as np

## First step is to load very large pickle file 
This file contains ~1.3 billion rows 

In [ ]:
OUT_PATH = "/shared/3/projects/newsDiffusion/data/interim/NEREmbedding/embeddingPairSimilarity2020.pkl"

In [2]:
#load pairs that were generated from entity clustering 
pairDf = pd.read_pickle("/shared/3/projects/newsDiffusion/data/interim/NEREmbedding/entityPairs2020.pkl")

In [11]:
def getCos(inList): 
    return 1 - distance.cosine(inList[0], inList[1])

def getCosSeries(inSeries): 
    return inSeries.apply(getCos)

#tqdm.pandas()
#exploded.head(1000000)["embedding"].progress_map(getCos)

embeddings = pairDf["embedding"]

print("starting similarity calculations")
with Pool(12) as pool: 
    print("splitting embedding list")
    splitList = np.array_split(embeddings, 10)
    
    print("getting similarity")
    similarityArrs = list(tqdm(pool.imap(getCosSeries, splitList), total=10))
    
    print("concatenating chunks together")
    similarity = pd.concat(similarityArrs)

print("adding column to dataframe")
pairDf = pairDf.drop(columns=["embedding"])
pairDf["similarity"] = similarity


starting similarity calculations


  0%|          | 0/10 [00:00<?, ?it/s]

<ipython-input-11-d3f92447073f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testDf["similarity"] = similarity


In [16]:
pairDf.to_pickle(OUT_PATH)

,ent_type,entity,key1,key2,similarity
0,event,america first,1727985,1728036,0.985507
1,event,america first,1727985,2003441,0.228275
2,event,america first,1727985,2003468,0.216420
3,event,america first,1727985,2613520,0.238758
4,event,america first,1727985,2660920,0.257908
...,...,...,...,...,...
50014,event,a civil war,2096287,2298235,0.177401
50015,event,a civil war,2096287,2331446,0.139892
50016,event,a civil war,2096287,2350830,0.168434
50017,event,a civil war,2096287,2362779,0.075460
